# 第二章 模型，提示和输出解释器

 - [一、设置OpenAI API Key](#一、设置OpenAI-API-Key)
 - [二、直接使用OpenAI](#二、直接使用OpenAI)
     - [2.1 计算1+1](#2.1-计算1+1)
     - [2.2 用美式英语表达海盗邮件](#2.2-用美式英语表达海盗邮件)
     - [2.3 中文版](#2.3-中文版)
 - [三、通过LangChain使用OpenAI](#三、通过LangChain使用OpenAI)
     - [3.1 模型](#3.1-模型)
     - [3.2 提示模板](#3.2-提示模板)
         - [3.2.1 使用LangChain提示模版](#3.2.1-使用LangChain提示模版)
         - [3.2.2 中文版](#3.2.2-中文版)
         - [3.2.2 为什么需要提示模版](#3.2.2-为什么需要提示模版)
     - [3.3 输出解析器](#3.3-输出解析器)
         - [3.3.1 如果没有输出解析器](#3.3.1-如果没有输出解析器)
         - [3.3.2 中文版](#3.3.2-中文版)
         - [3.3.3 LangChain输出解析器](#3.3.3-LangChain输出解析器)
         - [3.3.4 中文版](#3.3.4-中文版)
 - [四、补充材料](#四、补充材料)
     - [4.1 链式思考推理(ReAct)](#4.1-链式思考推理(ReAct))


## 一、设置OpenAI API Key

登陆 [OpenAI 账户](https://platform.openai.com/account/api-keys) 获取API Key，然后将其设置为环境变量。

- 如果你想要设置为全局环境变量，可以参考[知乎文章](https://zhuanlan.zhihu.com/p/627665725)。
- 如果你想要设置为本地/项目环境变量，在本文件目录下创建`.env`文件, 打开文件输入以下内容。

    <p style="font-family:verdana; font-size:12px;color:green">
    OPENAI_API_KEY="your_api_key" 
    </p>
  
  替换"your_api_key"为你自己的 API Key

In [2]:
# 下载需要的包python-dotenv和openai
# 如果你需要查看安装过程日志，可删除 -q 
!pip install -q python-dotenv
!pip install -q openai

In [3]:
import os
import openai
from dotenv import load_dotenv, find_dotenv

# 读取本地/项目的环境变量。

# find_dotenv()寻找并定位.env文件的路径
# load_dotenv()读取该.env文件，并将其中的环境变量加载到当前的运行环境中  
# 如果你设置的是全局的环境变量，这行代码则没有任何作用。
_ = load_dotenv(find_dotenv())

# 获取环境变量 OPENAI_API_KEY
import os
openai.api_key = os.environ.get("OPENAI_API_KEY")

deployment_id = None
API_TYPE=os.getenv("OPENAI_API_TYPE", None)
if not API_TYPE is None:
    openai.api_type=API_TYPE
    openai.api_base=os.environ["OPENAI_API_BASE"]
    openai.api_version=os.environ["OPENAI_API_VERSION"]
    deployment_id=os.getenv("DEPLOYMENT_ID", "gpt-35-turbo") 

## 二、直接使用OpenAI

我们先从直接调用OpenAI的API开始。

`get_completion`函数是基于`openai`的封装函数，对于给定提示（prompt）输出相应的回答。其包含两个参数
   
   - `prompt` 必需输入参数。 你给模型的**提示，可以是一个问题，可以是你需要模型帮助你做的事**（改变文本写作风格，翻译，回复消息等等）。
   - `model` 非必需输入参数。默认使用gpt-3.5-turbo。你也可以选择其他模型。
   
这里的提示对应我们给chatgpt的问题，函数给出的输出则对应chatpgt给我们的答案。

In [4]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    
    messages = [{"role": "user", "content": prompt}]
    
    response = openai.ChatCompletion.create(
        engine=deployment_id,
        model=model,
        messages=messages,
        temperature=0, 
    )
    return response.choices[0].message["content"]

### 2.1 计算1+1

我们来一个简单的例子 - 分别用中英文问问模型

- 中文提示(Prompt in Chinese)： `1+1是什么？`
- 英文提示(Prompt in English)： `What is 1+1?`

In [5]:
# 中文
get_completion("1+1是什么？")

'1+1等于2。'

In [ ]:
# 英文
get_completion("What is 1+1?")

### 2.2 用美式英语表达海盗邮件

上面的简单例子，模型`gpt-3.5-turbo`对我们的关于1+1是什么的提问给出了回答。

现在我们来看一个复杂一点的例子： 

假设我们是电商公司员工，我们的顾客是一名海盗A，他在我们的网站上买了一个榨汁机用来做奶昔，在制作奶昔的过程中，奶昔的盖子飞了出去，弄得厨房墙上到处都是。于是海盗A给我们的客服中心写来以下邮件：`customer_email`

In [8]:
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse,\
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

我们的客服人员对于海盗的措辞表达觉得有点难以理解。 现在我们想要实现两个小目标：

- 让模型用美式英语的表达方式将海盗的邮件进行翻译，客服人员可以更好理解。*这里海盗的英文表达可以理解为英文的方言，其与美式英语的关系，就如四川话与普通话的关系。
- 让模型在翻译是用平和尊重的语气进行表达，客服人员的心情也会更好。

根据这两个小目标，定义一下文本表达风格：`style`

In [9]:
# 美式英语 + 平静、尊敬的语调
style = """American English \
in a calm and respectful tone
"""

下一步需要做的是将`customer_email`和`style`结合起来构造我们的提示:`prompt`

In [10]:
# 要求模型根据给出的语调进行转化
prompt = f"""Translate the text \
that is delimited by triple backticks 
into a style that is {style}.
text: ```{customer_email}```
"""

print(prompt)

Translate the text that is delimited by triple backticks 
into a style that is American English in a calm and respectful tone
.
text: ```
Arrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse,the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!
```



`prompt` 构造好了，我们可以调用`get_completion`得到我们想要的结果 - 用平和尊重的语气，美式英语表达的海盗语言邮件

In [11]:
response = get_completion(prompt)

In [12]:
response

'I am quite frustrated that my blender lid flew off and made a mess of my kitchen walls with smoothie! To add to my frustration, the warranty does not cover the cost of cleaning up my kitchen. I kindly request your assistance at this moment, my friend.'

对比语言风格转换前后，用词更为正式，替换了极端情绪的表达，并表达了感谢。

✨ 你可以尝试修改提示，看可以得到什么不一样的结果😉

### 2.3 中文版

In [14]:
# 非正式用语
customer_email = """   
阿，我很生气，\
因为我的搅拌机盖掉了，\
把奶昔溅到了厨房的墙上！\
更糟糕的是，保修不包括打扫厨房的费用。\
我现在需要你的帮助，伙计！
"""

In [15]:
# 普通话 + 平静、尊敬的语调
style = """正式普通话 \
用一个平静、尊敬的语调
"""

In [16]:
# 要求模型根据给出的语调进行转化
prompt = f"""把由三个反引号分隔的文本\
翻译成一种{style}风格。
文本: ```{customer_email}```
"""

print(prompt)



把由三个反引号分隔的文本翻译成一种正式普通话 用一个平静、尊敬的语调
风格。
文本: ```   
阿，我很生气，因为我的搅拌机盖掉了，把奶昔溅到了厨房的墙上！更糟糕的是，保修不包括打扫厨房的费用。我现在需要你的帮助，伙计！
```



In [17]:
response = get_completion(prompt)

response

'尊敬的朋友们，我感到非常不安，因为我的搅拌机盖子不慎掉落，导致奶昔溅到了厨房的墙壁上！更加令人糟心的是，保修服务并不包含厨房清洁的费用。此刻，我真诚地请求各位的帮助，朋友们！'

## 三、通过LangChain使用OpenAI

在前面一部分，我们通过封装函数`get_completion`直接调用了OpenAI完成了对方言邮件进行了的翻译，得到用平和尊重的语气、正式的普通话表达的邮件。

让我们尝试使用LangChain来实现相同的功能。

In [21]:
# 如果你需要查看安装过程日志，可删除 -q 
# --upgrade 让我们可以安装到最新版本的 langchain
!pip install -q --upgrade langchain

### 3.1 模型

从`langchain.chat_models`导入`OpenAI`的对话模型`ChatOpenAI`。 除去OpenAI以外，`langchain.chat_models`还集成了其他对话模型，更多细节可以查看[Langchain官方文档](https://python.langchain.com/en/latest/modules/models/chat/integrations.html)。

In [18]:
from langchain.chat_models import AzureChatOpenAI

In [19]:
# 这里我们将参数temperature设置为0.0，从而减少生成答案的随机性。
# 如果你想要每次得到不一样的有新意的答案，可以尝试调整该参数。
chat = AzureChatOpenAI(
    openai_api_key=os.environ["OPENAI_API_KEY"],
    openai_api_type=os.environ["OPENAI_API_TYPE"],
    openai_api_base=os.environ["OPENAI_API_BASE"],
    openai_api_version=os.environ["OPENAI_API_VERSION"],
    deployment_name=deployment_id,
    temperature=0.0)
chat

AzureChatOpenAI(client=<class 'openai.api_resources.chat_completion.ChatCompletion'>, temperature=0.0, openai_api_key='584eb65f6a5346faa3d526aa977197ad', openai_api_base='https://myopenai4test.openai.azure.com/', deployment_name='gpt-35-turbo', openai_api_type='azure', openai_api_version='2023-05-15')

上面的输出显示ChatOpenAI的默认模型为`gpt-3.5-turbo`

### 3.2 提示模板

在前面的例子中，我们通过[f字符串](https://docs.python.org/zh-cn/3/tutorial/inputoutput.html#tut-f-strings)把Python表达式的值`style`和`customer_email`添加到`prompt`字符串内。

```python
prompt = f"""Translate the text \
that is delimited by triple backticks 
into a style that is {style}.
text: ```{customer_email}```
"""
```
`langchain`提供了接口方便快速的构造和使用提示。现在我们来看看如何使用`langchain`来构造提示。

#### 3.2.1 使用LangChain提示模版

##### 1️⃣ 构造提示模版字符串
我们构造一个提示模版字符串：`template_string`

In [20]:
template_string = """Translate the text \
that is delimited by triple backticks \
into a style that is {style}. \
text: ```{text}```
"""

##### 2️⃣ 构造LangChain提示模版
我们调用`ChatPromptTemplatee.from_template()`函数将上面的提示模版字符`template_string`转换为提示模版`prompt_template`

In [21]:
from langchain.prompts import ChatPromptTemplate
prompt_template = ChatPromptTemplate.from_template(template_string)

In [22]:
print(prompt_template.messages[0].prompt)

input_variables=['style', 'text'] template='Translate the text that is delimited by triple backticks into a style that is {style}. text: ```{text}```\n'


从上面的输出可以看出，`prompt_template` 有两个输入变量： `style` 和 `text`。

In [23]:
print(prompt_template.messages[0].prompt.input_variables)

['style', 'text']


##### 3️⃣ 使用模版得到客户消息提示

langchain提示模版`prompt_template`需要两个输入变量： `style` 和 `text`。 这里分别对应 
- `customer_style`: 我们想要的顾客邮件风格
- `customer_email`: 顾客的原始邮件文本。

In [24]:
customer_style = """American English \
in a calm and respectful tone
"""

In [25]:
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse, \
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

对于给定的`customer_style`和`customer_email`, 我们可以使用提示模版`prompt_template`的`format_messages`方法生成想要的客户消息`customer_messages`。

In [26]:
customer_messages = prompt_template.format_messages(
                    style=customer_style,
                    text=customer_email)

In [27]:
print(type(customer_messages))
print(type(customer_messages[0]))

<class 'list'>
<class 'langchain.schema.messages.HumanMessage'>


可以看出`customer_messages`变量类型为列表(`list`)，而列表里的元素变量类型为langchain自定义消息(`langchain.schema.HumanMessage`)。

打印第一个元素可以得到如下:

In [28]:
print(customer_messages[0])

content="Translate the text that is delimited by triple backticks into a style that is American English in a calm and respectful tone\n. text: ```\nArrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse, the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!\n```\n"


##### 4️⃣ 调用chat模型转换客户消息风格

现在我们可以调用[模型](#model)部分定义的chat模型来实现转换客户消息风格。到目前为止，我们已经实现了在前一部分的任务。

In [29]:
customer_response = chat(customer_messages)

In [30]:
print(customer_response.content)

I'm really frustrated that my blender lid flew off and made a mess of my kitchen walls with smoothie! And to make things even worse, the warranty doesn't cover the cost of cleaning up my kitchen. I could really use your help right now, my friend!


In [31]:
print(customer_response.content)

I'm really frustrated that my blender lid flew off and made a mess of my kitchen walls with smoothie! And to make things even worse, the warranty doesn't cover the cost of cleaning up my kitchen. I could really use your help right now, my friend!


##### 5️⃣ 使用模版得到回复消息提示

接下来，我们更进一步，将客服人员回复的消息，转换为海盗的语言风格，并确保消息比较有礼貌。 

这里，我们可以继续使用第2️⃣步构造的langchain提示模版，来获得我们回复消息提示。

In [32]:
service_reply = """Hey there customer, \
the warranty does not cover \
cleaning expenses for your kitchen \
because it's your fault that \
you misused your blender \
by forgetting to put the lid on before \
starting the blender. \
Tough luck! See ya!
"""

In [33]:
service_style_pirate = """\
a polite tone \
that speaks in English Pirate\
"""

In [34]:
service_messages = prompt_template.format_messages(
    style=service_style_pirate,
    text=service_reply)

print(service_messages[0].content)

Translate the text that is delimited by triple backticks into a style that is a polite tone that speaks in English Pirate. text: ```Hey there customer, the warranty does not cover cleaning expenses for your kitchen because it's your fault that you misused your blender by forgetting to put the lid on before starting the blender. Tough luck! See ya!
```



##### 6️⃣ 调用chat模型转换回复消息风格

调用[模型](#model)部分定义的chat模型来转换回复消息风格

In [35]:
service_response = chat(service_messages)
print(service_response.content)

Ahoy there, matey! I regret to inform ye that the warranty be not coverin' the costs o' cleanin' yer galley, as 'tis yer own fault fer misusin' yer blender by forgettin' to secure the lid afore startin' it. Aye, tough luck, me heartie! Fare thee well!


In [36]:
# 中文提示
from langchain.prompts import ChatPromptTemplate

template_string = """把由三个反引号分隔的文本\
翻译成一种{style}风格。\
文本: ```{text}```
"""
prompt_template = ChatPromptTemplate.from_template(template_string)

customer_style = """正式普通话 \
用一个平静、尊敬的语气
"""

customer_email = """
阿，我很生气，\
因为我的搅拌机盖掉了，\
把奶昔溅到了厨房的墙上！\
更糟糕的是，保修不包括打扫厨房的费用。\
我现在需要你的帮助，伙计！
"""

customer_messages = prompt_template.format_messages(
                    style=customer_style,
                    text=customer_email)


print(customer_messages[0])

content='把由三个反引号分隔的文本翻译成一种正式普通话 用一个平静、尊敬的语气\n风格。文本: ```\n阿，我很生气，因为我的搅拌机盖掉了，把奶昔溅到了厨房的墙上！更糟糕的是，保修不包括打扫厨房的费用。我现在需要你的帮助，伙计！\n```\n'


In [37]:
service_reply = """嘿，顾客， \
保修不包括厨房的清洁费用， \
因为您在启动搅拌机之前 \
忘记盖上盖子而误用搅拌机, \
这是您的错。 \
倒霉！ 再见！
"""

service_style_pirate = """\
一个有礼貌的语气 \
使用正式的普通话 \
"""
service_messages = prompt_template.format_messages(
    style=service_style_pirate,
    text=service_reply)

print(service_messages[0].content)

把由三个反引号分隔的文本翻译成一种一个有礼貌的语气 使用正式的普通话 风格。文本: ```嘿，顾客， 保修不包括厨房的清洁费用， 因为您在启动搅拌机之前 忘记盖上盖子而误用搅拌机, 这是您的错。 倒霉！ 再见！
```



In [38]:
service_response = chat(service_messages)
print(service_response.content)

尊敬的顾客，很抱歉告知您，保修服务不包含厨房清洁费用。这是因为在您使用搅拌机之前，遗忘了盖上盖子而导致了误用。我们希望您能理解，这是一个不幸的疏忽。再次为此带来的不便向您致以诚挚的歉意。祝您一切顺利，再见。


#### 3.2.2 为什么需要提示模版

在应用于比较复杂的场景时，提示可能会非常长并且包含涉及许多细节。**使用提示模版，可以让我们更为方便地重复使用设计好的提示**。

下面给出了一个比较长的提示模版案例。学生们线上学习并提交作业，通过以下的提示来实现对学生的提交的作业的评分。

In [40]:
# 英文版
prompt = """ Your task is to determine if the student's solution is correct or not

    To solve the problem do the following:
    - First, workout your own solution to the problem
    - Then compare your solution to the student's solution 
    and evaluate if the sudtent's solution is correct or not.
    ...
    Use the following format:
    Question:
    ```
    question here
    ```
    Student's solution:
    ```
    student's solution here
    ```
    Actual solution:
    ```
    ...
    steps to work out the solution and your solution here
    ```
    Is the student's solution the same as acutal solution \
    just calculated:
    ```
    yes or no
    ```
    Student grade
    ```
    correct or incorrect
    ```
    
    Question:
    ```
    {question}
    ```
    Student's solution:
    ```
    {student's solution}
    ```
    Actual solution:
    
    """

此外，LangChain还提供了提示模版用于一些常用场景。比如自动摘要、问答、连接到SQL数据库、连接到不同的API. 通过使用LongChain内置的提示模版，你可以快速建立自己的大模型应用，而不需要花时间去设计和构造提示。

最后，我们在建立大模型应用时，通常希望模型的输出为给定的格式，比如在输出使用特定的关键词来让输出结构化。 下面为一个使用大模型进行链式思考推理例子，对于问题：*What is the elevation range for the area that the eastern sector of the Colorado orogeny extends into?*

通过使用LangChain库函数，输出采用"Thought"（思考）、"Action"（行动）、"Observation"（观察）作为链式思考推理的关键词，让输出结构化。

```
Thought: I need to search Colorado orogeny, find the area that the eastern sector of the Colorado orogeny extends into, then find the elevation range of the area.
Action: Search[Colorado orogeny]
Observation: The Colorado orogeny was an episode of mountain building (an orogeny) in Colorado and surrounding areas.

Thought: It does not mention the eastern sector. So I need to look up eastern sector.
Action: Lookup[eastern sector]
Observation: (Result 1 / 1) The eastern sector extends into the High Plains and is called the Central Plains orogeny.

Thought: The eastern sector of Colorado orogeny extends into the High Plains. So I need to search High Plains and find its elevation range.
Action: Search[High Plains]
Observation: High Plains refers to one of two distinct land regions

Thought: I need to instead search High Plains (United States).
Action: Search[High Plains (United States)]
Observation: The High Plains are a subregion of the Great Plains. From east to west, the High Plains rise in elevation from around 1,800 to 7,000 ft (550 to 2,130 m).[3]

Thought: High Plains rise in elevation from around 1,800 to 7,000 ft, so the answer is 1,800 to 7,000 ft.
Action: Finish[1,800 to 7,000 ft]
```

在补充材料中，可以查看使用LangChain和OpenAI进行链式思考推理的另一个代码实例。

In [41]:
# 中文版
prompt = """ 你的任务是判断学生的解决方案是正确的还是不正确的

要解决该问题，请执行以下操作：
 - 首先，制定自己的问题解决方案
 - 然后将您的解决方案与学生的解决方案进行比较
 并评估学生的解决方案是否正确。
...
使用下面的格式:

问题:
```
问题文本
```
学生的解决方案:
```
学生的解决方案文本
```
实际解决方案:
```
...
制定解决方案的步骤以及您的解决方案请参见此处
```
学生的解决方案和实际解决方案是否相同 \
只计算：
```
是或者不是
```
学生的成绩
```
正确或者不正确
```

问题:
```
{question}
```
学生的解决方案:
```
{student's solution}
```
实际解决方案:

"""

### 3.3 输出解析器

#### 3.3.1 如果没有输出解析器

对于给定的评价`customer_review`, 我们希望提取信息，并按以下格式输出：

```python
{
  "gift": False,
  "delivery_days": 5,
  "price_value": "pretty affordable!"
}
```

In [62]:
customer_review = """\
This leaf blower is pretty amazing.  It has four settings:\
candle blower, gentle breeze, windy city, and tornado. \
It arrived in two days, just in time for my wife's \
anniversary present. \
I think my wife liked it so much she was speechless. \
So far I've been the only one using it, and I've been \
using it every other morning to clear the leaves on our lawn. \
It's slightly more expensive than the other leaf blowers \
out there, but I think it's worth it for the extra features.
"""

##### 1️⃣ 构造提示模版字符串

In [63]:
review_template = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product \
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

Format the output as JSON with the following keys:
gift
delivery_days
price_value

text: {text}
"""

##### 2️⃣ 构造langchain提示模版

In [64]:
from langchain.prompts import ChatPromptTemplate
prompt_template = ChatPromptTemplate.from_template(review_template)
print(prompt_template)

input_variables=['text'] messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], template='For the following text, extract the following information:\n\ngift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.\n\ndelivery_days: How many days did it take for the product to arrive? If this information is not found, output -1.\n\nprice_value: Extract any sentences about the value or price,and output them as a comma separated Python list.\n\nFormat the output as JSON with the following keys:\ngift\ndelivery_days\nprice_value\n\ntext: {text}\n'))]


##### 3️⃣ 使用模版得到提示消息

In [65]:
messages = prompt_template.format_messages(text=customer_review)

##### 4️⃣ 调用chat模型提取信息

In [66]:
response = chat(messages)
print(response.content)

{
  "gift": false,
  "delivery_days": 2,
  "price_value": ["It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."]
}


##### 📝 分析与总结
`response.content`类型为字符串（`str`），而并非字典(`dict`), 直接使用`get`方法会报错。因此，我们需要输出解释器。

In [67]:
type(response.content)

str

In [68]:
response.content.get('gift')

AttributeError: 'str' object has no attribute 'get'

#### 3.3.2 中文版

In [69]:
from langchain.prompts import ChatPromptTemplate

customer_review_zh = """\
这款吹叶机非常神奇。 它有四个设置：\
吹蜡烛、微风、风城、龙卷风。 \
两天后就到了，正好赶上我妻子的\
周年纪念礼物。 \
我想我的妻子会喜欢它到说不出话来。 \
到目前为止，我是唯一一个使用它的人，而且我一直\
每隔一天早上用它来清理草坪上的叶子。 \
它比其他吹叶机稍微贵一点，\
但我认为它的额外功能是值得的。
"""

review_template_zh = """\
对于以下文本，请从中提取以下信息：

礼物：该商品是作为礼物送给别人的吗？ \
如果是，则回答 是的；如果否或未知，则回答 不是。

交货天数：产品需要多少天\
到达？ 如果没有找到该信息，则输出-1。

价钱：提取有关价值或价格的任何句子，\
并将它们输出为逗号分隔的 Python 列表。

使用以下键将输出格式化为 JSON：
礼物
交货天数
价钱

文本: {text}
"""

prompt_template = ChatPromptTemplate.from_template(review_template_zh)
print(prompt_template)


input_variables=['text'] messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], template='对于以下文本，请从中提取以下信息：\n\n礼物：该商品是作为礼物送给别人的吗？ 如果是，则回答 是的；如果否或未知，则回答 不是。\n\n交货天数：产品需要多少天到达？ 如果没有找到该信息，则输出-1。\n\n价钱：提取有关价值或价格的任何句子，并将它们输出为逗号分隔的 Python 列表。\n\n使用以下键将输出格式化为 JSON：\n礼物\n交货天数\n价钱\n\n文本: {text}\n'))]


In [70]:
messages = prompt_template.format_messages(text=customer_review_zh)

response = chat(messages)
print(response.content)

{
  "礼物": "是的",
  "交货天数": 2,
  "价钱": ["它比其他吹叶机稍微贵一点"]
}


#### 3.3.3 LangChain输出解析器

##### 1️⃣ 构造提示模版字符串

In [71]:
review_template_2 = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product\
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

text: {text}

{format_instructions}
"""

##### 2️⃣ 构造langchain提示模版

In [72]:
prompt = ChatPromptTemplate.from_template(template=review_template_2)

##### 🔥 构造输出解析器

In [95]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

gift_schema = ResponseSchema(name="gift",
                             description="Was the item purchased\
                             as a gift for someone else? \
                             Answer True if yes,\
                             False if not or unknown.")

delivery_days_schema = ResponseSchema(name="delivery_days",
                                      type="number",
                                      description="How many days\
                                      did it take for the product\
                                      to arrive? If this \
                                      information is not found,\
                                      output -1.")

price_value_schema = ResponseSchema(name="price_value",
                                    description="Extract any\
                                    sentences about the value or \
                                    price, and output them as a \
                                    comma separated Python list.")


response_schemas = [gift_schema, 
                    delivery_days_schema,
                    price_value_schema]
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions()
print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"gift": string  // Was the item purchased                             as a gift for someone else?                              Answer True if yes,                             False if not or unknown.
	"delivery_days": number  // How many days                                      did it take for the product                                      to arrive? If this                                       information is not found,                                      output -1.
	"price_value": string  // Extract any                                    sentences about the value or                                     price, and output them as a                                     comma separated Python list.
}
```


##### 3️⃣ 使用模版得到提示消息

In [96]:
messages = prompt.format_messages(text=customer_review, format_instructions=format_instructions)

In [97]:
print(messages[0].content)

对于以下文本，请从中提取以下信息：：

礼物：该商品是作为礼物送给别人的吗？
如果是，则回答 是的；如果否或未知，则回答 不是。

交货天数：产品到达需要多少天？ 如果没有找到该信息，则输出-1。

价钱：提取有关价值或价格的任何句子，并将它们输出为逗号分隔的 Python 列表。

文本: This leaf blower is pretty amazing.  It has four settings:candle blower, gentle breeze, windy city, and tornado. It arrived in two days, just in time for my wife's anniversary present. I think my wife liked it so much she was speechless. So far I've been the only one using it, and I've been using it every other morning to clear the leaves on our lawn. It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features.


The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"gift": string  // Was the item purchased                             as a gift for someone else?                              Answer True if yes,                             False if not or unknown.
	"delivery_days": number  // Ho

##### 4️⃣ 调用chat模型提取信息

In [98]:
response = chat(messages)
print(response.content)

```json
{
	"gift": false,
	"delivery_days": 2,
	"price_value": "It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."
}
```


##### 5️⃣ 使用输出解析器解析输出

In [99]:
output_dict = output_parser.parse(response.content)
output_dict

{'gift': False,
 'delivery_days': 2,
 'price_value': "It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."}

##### 📝 分析与总结
`output_dict`类型为字典(`dict`), 可直接使用`get`方法。这样的输出更方便下游任务的处理。

In [100]:
type(output_dict)

dict

In [101]:
output_dict.get('delivery_days')

2

#### 3.3.4 中文版

In [102]:
# 中文
review_template_2 = """\
对于以下文本，请从中提取以下信息：：

礼物：该商品是作为礼物送给别人的吗？
如果是，则回答 是的；如果否或未知，则回答 不是。

交货天数：产品到达需要多少天？ 如果没有找到该信息，则输出-1。

价钱：提取有关价值或价格的任何句子，并将它们输出为逗号分隔的 Python 列表。

文本: {text}

{format_instructions}
"""

from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

gift_schema = ResponseSchema(name="礼物",
                             description="这件物品是作为礼物送给别人的吗？\
                            如果是，则回答 是的，\
                            如果否或未知，则回答 不是。")

delivery_days_schema = ResponseSchema(name="交货天数",
                                      type="number",
                                      description="产品需要多少天才能到达？\
                                      如果没有找到该信息，则输出-1。")

price_value_schema = ResponseSchema(name="价钱",
                                    description="提取有关价值或价格的任何句子，\
                                    并将它们输出为逗号分隔的 Python 列表")


response_schemas = [gift_schema, 
                    delivery_days_schema,
                    price_value_schema]
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions()
print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"礼物": string  // 这件物品是作为礼物送给别人的吗？                            如果是，则回答 是的，                            如果否或未知，则回答 不是。
	"交货天数": number  // 产品需要多少天才能到达？                                      如果没有找到该信息，则输出-1。
	"价钱": string  // 提取有关价值或价格的任何句子，                                    并将它们输出为逗号分隔的 Python 列表
}
```


In [103]:
prompt = ChatPromptTemplate.from_template(template=review_template_2)
messages = prompt.format_messages(text=customer_review_zh, format_instructions=format_instructions)
print(messages[0].content)

对于以下文本，请从中提取以下信息：：

礼物：该商品是作为礼物送给别人的吗？
如果是，则回答 是的；如果否或未知，则回答 不是。

交货天数：产品到达需要多少天？ 如果没有找到该信息，则输出-1。

价钱：提取有关价值或价格的任何句子，并将它们输出为逗号分隔的 Python 列表。

文本: 这款吹叶机非常神奇。 它有四个设置：吹蜡烛、微风、风城、龙卷风。 两天后就到了，正好赶上我妻子的周年纪念礼物。 我想我的妻子会喜欢它到说不出话来。 到目前为止，我是唯一一个使用它的人，而且我一直每隔一天早上用它来清理草坪上的叶子。 它比其他吹叶机稍微贵一点，但我认为它的额外功能是值得的。


The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"礼物": string  // 这件物品是作为礼物送给别人的吗？                            如果是，则回答 是的，                            如果否或未知，则回答 不是。
	"交货天数": number  // 产品需要多少天才能到达？                                      如果没有找到该信息，则输出-1。
	"价钱": string  // 提取有关价值或价格的任何句子，                                    并将它们输出为逗号分隔的 Python 列表
}
```



In [104]:
response = chat(messages)
print(response.content)


```json
{
	"礼物": "不是",
	"交货天数": 2,
	"价钱": "它比其他吹叶机稍微贵一点"
}
```


In [105]:
output_dict = output_parser.parse(response.content)
output_dict

{'礼物': '不是', '交货天数': 2, '价钱': '它比其他吹叶机稍微贵一点'}

## 四、补充材料

### 4.1 链式思考推理(ReAct)
参考资料：[ReAct (Reason+Act) prompting in OpenAI GPT and LangChain](https://tsmatz.wordpress.com/2023/03/07/react-with-openai-gpt-and-langchain/)

In [106]:
!pip install -q wikipedia

In [111]:
from langchain.docstore.wikipedia import Wikipedia
from langchain.llms import AzureOpenAI
from langchain.agents import initialize_agent, Tool, AgentExecutor
from langchain.agents.react.base import DocstoreExplorer

proxy = 'socks5://127.0.0.1:13659'
os.environ['http_proxy'] = proxy 
os.environ['HTTP_PROXY'] = proxy
os.environ['https_proxy'] = proxy
os.environ['HTTPS_PROXY'] = proxy


docstore=DocstoreExplorer(Wikipedia())
tools = [
  Tool(
    name="Search",
    func=docstore.search,
    description="Search for a term in the docstore.",
  ),
  Tool(
    name="Lookup",
    func=docstore.lookup,
    description="Lookup a term in the docstore.",
  )
]

# 使用大语言模型
llm = AzureOpenAI(
  deployment_name="gpt-35-turbo-instruct",
  temperature=0,
)

# 初始化ReAct代理
react = initialize_agent(tools, llm, agent="react-docstore", verbose=True)
agent_executor = AgentExecutor.from_agent_and_tools(
  agent=react.agent,
  tools=tools,
  verbose=True,
)


question = "Author David Chanoff has collaborated with a U.S. Navy admiral who served as the ambassador to the United Kingdom under which President?"
agent_executor.run(question)



> Entering new AgentExecutor chain...
Thought: I need to search David Chanoff, find his collaborations, then find the U.S. Navy admiral he collaborated with. After that, I need to find the president under whom the admiral served as the ambassador to the United Kingdom.
Action: Search[David Chanoff]
Observation: David Chanoff is a noted author of non-fiction work. His work has typically involved collaborations with the principal protagonist of the work concerned. His collaborators have included; Augustus A. White, Joycelyn Elders, Đoàn Văn Toại, William J. Crowe, Ariel Sharon, Kenneth Good and Felix Zandman. He has also written about a wide range of subjects including literary history, education and foreign for The Washington Post, The New Republic and The New York Times Magazine. He has published more than twelve books.
Thought: David Chanoff has collaborated with Augustus A. White, Joycelyn Elders, Đoàn Văn Toại, William J. Crowe, Ariel Sharon, Kenneth Good, and Felix Zandman. I nee

KeyboardInterrupt: 